In [1]:
import netCDF4 as nc
import numpy as np
import datetime as dt
import shutil

In [2]:
origfile='/results/forcing/atmospheric/GEM2.5/gemlam/gemlam_y2010m04d01.nc'
destfile='/data/eolson/results/MEOPAR/GEMLAM/gemlam_y2010m04d01.nc'
date0=dt.datetime(2010,4,1)

In [3]:
fold=nc.Dataset(origfile,'r')

In [4]:
[dt.datetime(1950,1,1)+dt.timedelta(seconds=ii) for ii 
      in fold.variables['time_counter'][:]]

<ipython-input-4-0d9c439d0c95>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  in fold.variables['time_counter'][:]]


[datetime.datetime(2010, 3, 29, 0, 0),
 datetime.datetime(2010, 3, 29, 1, 0),
 datetime.datetime(2010, 3, 29, 2, 0),
 datetime.datetime(2010, 3, 29, 3, 0),
 datetime.datetime(2010, 3, 29, 4, 0),
 datetime.datetime(2010, 3, 29, 5, 0),
 datetime.datetime(2010, 3, 29, 6, 0),
 datetime.datetime(2010, 3, 29, 7, 0),
 datetime.datetime(2010, 3, 29, 8, 0),
 datetime.datetime(2010, 3, 29, 9, 0),
 datetime.datetime(2010, 3, 29, 10, 0),
 datetime.datetime(2010, 3, 29, 11, 0),
 datetime.datetime(2010, 3, 29, 12, 0),
 datetime.datetime(2010, 3, 31, 1, 0),
 datetime.datetime(2010, 4, 1, 14, 0),
 datetime.datetime(2010, 4, 1, 15, 0),
 datetime.datetime(2010, 4, 1, 16, 0),
 datetime.datetime(2010, 4, 1, 17, 0),
 datetime.datetime(2010, 4, 1, 18, 0),
 datetime.datetime(2010, 4, 1, 19, 0),
 datetime.datetime(2010, 4, 1, 20, 0),
 datetime.datetime(2010, 4, 1, 21, 0),
 datetime.datetime(2010, 4, 1, 22, 0),
 datetime.datetime(2010, 4, 1, 23, 0)]

In [5]:
fnew=nc.Dataset(destfile,'w')

In [6]:
ttype=fold.variables['time_counter'].dtype

In [7]:
newcounter=[(date0-dt.datetime(1950,1,1)).total_seconds()+
           3600*ii for ii in range(0,24)]

In [8]:
fold.variables['time_counter'].dimensions

('time_counter',)

In [9]:
for name, dimension in fold.dimensions.items():
    fnew.createDimension(name, (len(dimension) if not dimension.isunlimited() else None))

In [10]:
fnew.createVariable('time_counter',ttype,('time_counter',),fill_value=fold['time_counter'].getncattr('_FillValue'))
fnew['time_counter'][:]=newcounter[:]

In [11]:
atlist=[el for el in fold['time_counter'].ncattrs() if el not in ('_FillValue',)]
atlist

['axis',
 'calendar',
 'cell_methods',
 'long_name',
 'standard_name',
 'time_origin',
 'title',
 'units']

In [12]:
for el in atlist:
    fnew['time_counter'].setncattr(el,fold['time_counter'].getncattr(el))

In [13]:
for name, variable in fold.variables.items():
        if name not in ('time_counter',):
            x = fnew.createVariable(name, variable.datatype, variable.dimensions,
                                    fill_value=variable.getncattr('_FillValue'))
            fnew[name][:] = fold[name][:]
            atlist=[el for el in variable.ncattrs() if el not in ('_FillValue',)]
            for el in atlist:
                fnew[name].setncattr(el,variable.getncattr(el))

<ipython-input-13-90da30461376>:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fnew[name][:] = fold[name][:]


In [14]:
fnew.history_of_appended_files=fold.history_of_appended_files

In [15]:
fnew.history=f'{dt.datetime.now()}: revised time_counter to accurately reflect day and time using Python netCDF4 \n'+fold.history

In [16]:
fnew.NCO=fold.NCO

In [17]:
fnew.nco_openmp_thread_number=fold.nco_openmp_thread_number

In [18]:
fnew.close()
fold.close()

In [19]:
f=nc.Dataset(destfile)

In [20]:
f

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    history_of_appended_files: Sat Oct 12 17:31:48 2019: Appended file /dev/shm/precip.nc had following "history" attribute:
Sat Oct 12 17:31:48 2019: /usr/bin/ncdiff -4 -O -o /dev/shm/precip.nc -v precip /data/dlatorne/tmp-rpn-to-gem-lam/gemlam_y2010m04d01_000.nc /data/dlatorne/tmp-rpn-to-gem-lam/gemlam_y2010m03d31_023.nc
Sat Oct 12 17:31:08 2019: /usr/bin/ncatted -a missing_variables,global,d,, /data/dlatorne/tmp-rpn-to-gem-lam/gemlam_y2010m04d01_000.nc
Sat Oct 12 17:31:08 2019: /usr/bin/ncks -4 -A -v solar /dev/shm/solar.nc /data/dlatorne/tmp-rpn-to-gem-lam/gemlam_y2010m04d01_000.nc
Sat Oct 12 17:31:08 2019: /usr/bin/ncks -4 -O -x -v solar /data/dlatorne/tmp-rpn-to-gem-lam/gemlam_y2010m04d01_000.nc /data/dlatorne/tmp-rpn-to-gem-lam/gemlam_y2010m04d01_000.nc
Sat Oct 12 17:12:53 2019: /usr/bin/ncap2 -4 -O -s RT=RT*1000 /data/dlatorne/tmp-rpn-to-gem-lam/2010033112_012.nc /data/dlatorne/tmp-rpn-to-gem-

In [21]:
f.variables

{'time_counter': <class 'netCDF4._netCDF4.Variable'>
 float64 time_counter(time_counter)
     _FillValue: nan
     axis: T
     calendar: proleptic_gregorian
     cell_methods: time_counter: mean
     long_name: Time axis
     standard_name: time
     time_origin: 1950-01-01 00:00:00
     title: Time
     units: seconds since 1950-01-01
 unlimited dimensions: time_counter
 current shape = (24,)
 filling on,
 'PRATE_surface': <class 'netCDF4._netCDF4.Variable'>
 float32 PRATE_surface(time_counter, y, x)
     _FillValue: nan
     level: surface
     long_name: Precipitation Rate
     standard_name: precipitation_flux
     units: kg/m^2/s
 unlimited dimensions: time_counter
 current shape = (24, 266, 256)
 filling on,
 'RH_2maboveground': <class 'netCDF4._netCDF4.Variable'>
 float32 RH_2maboveground(time_counter, y, x)
     _FillValue: nan
     level: 2 m above surface
     long_name: Relative Humidity
     standard_name: relative_humidity_2maboveground
     units: percent
     comment: c

In [22]:
[dt.datetime(1950,1,1)+dt.timedelta(seconds=ii) for ii 
      in f.variables['time_counter'][:]]

<ipython-input-22-6bbf72d9b8e1>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  in f.variables['time_counter'][:]]


[datetime.datetime(2010, 4, 1, 0, 0),
 datetime.datetime(2010, 4, 1, 1, 0),
 datetime.datetime(2010, 4, 1, 2, 0),
 datetime.datetime(2010, 4, 1, 3, 0),
 datetime.datetime(2010, 4, 1, 4, 0),
 datetime.datetime(2010, 4, 1, 5, 0),
 datetime.datetime(2010, 4, 1, 6, 0),
 datetime.datetime(2010, 4, 1, 7, 0),
 datetime.datetime(2010, 4, 1, 8, 0),
 datetime.datetime(2010, 4, 1, 9, 0),
 datetime.datetime(2010, 4, 1, 10, 0),
 datetime.datetime(2010, 4, 1, 11, 0),
 datetime.datetime(2010, 4, 1, 12, 0),
 datetime.datetime(2010, 4, 1, 13, 0),
 datetime.datetime(2010, 4, 1, 14, 0),
 datetime.datetime(2010, 4, 1, 15, 0),
 datetime.datetime(2010, 4, 1, 16, 0),
 datetime.datetime(2010, 4, 1, 17, 0),
 datetime.datetime(2010, 4, 1, 18, 0),
 datetime.datetime(2010, 4, 1, 19, 0),
 datetime.datetime(2010, 4, 1, 20, 0),
 datetime.datetime(2010, 4, 1, 21, 0),
 datetime.datetime(2010, 4, 1, 22, 0),
 datetime.datetime(2010, 4, 1, 23, 0)]

In [23]:
f.close()

In [24]:
import xarray as xr

In [25]:
flist=['/data/eolson/results/MEOPAR/GEMLAM/gemlam_y2010m02d05.nc',
       '/data/eolson/results/MEOPAR/GEMLAM/gemlam_y2010m02d06.nc',
       '/data/eolson/results/MEOPAR/GEMLAM/gemlam_y2010m02d07.nc',
       '/data/eolson/results/MEOPAR/GEMLAM/gemlam_y2010m02d08.nc']

In [26]:
test=xr.open_mfdataset(flist)

In [27]:
test.close

<bound method DataWithCoords.close of <xarray.Dataset>
Dimensions:           (time_counter: 96, x: 256, y: 266)
Coordinates:
  * time_counter      (time_counter) datetime64[ns] 2010-02-05 ... 2010-02-08...
Dimensions without coordinates: x, y
Data variables:
    nav_lon           (time_counter, y, x) float32 dask.array<chunksize=(24, 266, 256), meta=np.ndarray>
    nav_lat           (time_counter, y, x) float32 dask.array<chunksize=(24, 266, 256), meta=np.ndarray>
    qair              (time_counter, y, x) float32 dask.array<chunksize=(24, 266, 256), meta=np.ndarray>
    RH_2maboveground  (time_counter, y, x) float32 dask.array<chunksize=(24, 266, 256), meta=np.ndarray>
    therm_rad         (time_counter, y, x) float32 dask.array<chunksize=(24, 266, 256), meta=np.ndarray>
    u_wind            (time_counter, y, x) float32 dask.array<chunksize=(24, 266, 256), meta=np.ndarray>
    v_wind            (time_counter, y, x) float32 dask.array<chunksize=(24, 266, 256), meta=np.ndarray>
    at